# feature engineering
Start:
* one database with two collections that hold all the sprints and issues with all the raw data
* only addition to this raw data is a field which repository they belong to

End:

* Sprints:
  * Make aggregation of each sprint's metadata:
    * sprint planned duration in days
    * sprint overdue boolean (completedDate > endDate)
    * Number of issues inside sprint
    * Median number of issues with changed sprint
* Issues:
  * predictor: has sprint field in change log (boolean)
  * Number of sprint fields in change log (int)
  * Add originSprintId field (uuid)
  * Add above sprint metadata fields to every issue
  * All not used (custom)fields should be removed

In [8]:
# Auxiliary
import pymongo
import re
import json
import numpy as np
from datetime import datetime
from dateutil import parser

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']

In [11]:
# Sprints
coll_issues = db['01_Issues']
coll_sprints = db['02_Sprints']
sprints = coll_sprints.find()

for sprint in sprints:
    # sprint planned duration in days
    try:
        startDate = parser.parse(sprint['startDate'])
        endDate = parser.parse(sprint['endDate'])
        plannedDuration =  (endDate - startDate).days
    except:
        plannedDuration = np.nan

    # Number of issues inside sprint
    issues = sprint['issues']
    noOfIssues = len(issues)
    changes_in_issues = 0
    sprint_changes_in_issues = 0
    
    # Get total number of changes and sprint changes 
    for issue in issues:
        doc = coll_issues.find_one({"id": issue})
        changes_in_issues += doc['totalChanges']
        sprint_changes_in_issues += doc['sprintChanges']

    coll_sprints.update_one({'id': sprint['id']}, {'$set': {'plannedDuration': plannedDuration, 'noOfIssues': noOfIssues, "totalChanges": changes_in_issues, "sprintChanges": sprint_changes_in_issues}})



In [ ]:
# Add above sprint metadata fields to every issue

In [ ]:
# Keep only a selection of fields as features
fields_to_keep = ['description', 'summary', 'resolution.name', 'priority.name', 'aggregatetimeoriginalestimate', 'timeestimate', 'assignnee.key', 'status.name', 'votes.votes', 'issuetype.name', 'project.id', 'watches.watchcount', 'comments.count']

In [ ]:
def get_origin_sprint(issue_id):
    sprint_id = re.search(r'id=(.*?),', issue_id[]).group(1)

In [ ]:
# Sprints that need the new Issue data
# Median number of issues with changed sprint